## make sure you have all libraries installed. 
use conda environment: /scratch/da2734/twitter/worldbank_twitter_environment.yml

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# loading filtered samples

In [1]:
# filtered contains 0.8G of data!!
import time
start_time = time.time()
import pyarrow.parquet as pq
from glob import glob
import os
country_code = 'US'
month = '2012-1'
path_to_data = '/scratch/spf248/twitter/data/classification/US/filtered/'
tweets_filtered=pq.ParquetDataset(glob(os.path.join(path_to_data,                                           
#                                            country_code,
#                                            month,
                                           '*.parquet'))).read().to_pandas()
tweets_filtered['tweet_id'] = tweets_filtered['tweet_id'].astype(int)
print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds')
print(tweets_filtered.shape)

time taken to load filtered sample: 309.1688368320465 seconds
(92121093, 11)


In [2]:
start_time = time.time()
tweets_filtered = tweets_filtered[['tweet_id', 'text']]
print('drop columns:', str(time.time() - start_time), 'seconds')

drop columns: 6.185930490493774 seconds


In [3]:
tweets_filtered.head()

,tweet_id,text
0,276933934124765184,Damn i have to much homework
1,277143632232992768,Does a bedazzler work on leather? Serious ques...
2,277150490276540416,RT @porcelain10: Washington Post D Milbank bas...
3,277151157208637440,Finally off work
4,277175194454462465,Irrational: No BioShock PS Vita until Sony and...


# loading random samples

In [1]:
# random contains 0.8G of data!!
import time
start_time = time.time()
import pyarrow.parquet as pq
from glob import glob
import os
country_code = 'US'
month = '2012-1'
path_to_data = '/scratch/spf248/twitter/data/classification/US/random/'
tweets_random=pq.ParquetDataset(glob(os.path.join(path_to_data,                                           
#                                            country_code,
#                                            month,
                                           '*.parquet'))).read().to_pandas()
tweets_random['tweet_id'] = tweets_random['tweet_id'].astype(int)
print('time taken to load random sample:', str(time.time() - start_time), 'seconds')
print(tweets_random.shape)

time taken to load random sample: 354.7586364746094 seconds
(92114009, 11)


In [2]:
start_time = time.time()
tweets_random = tweets_random[['tweet_id', 'text']]
print('drop columns:', str(time.time() - start_time), 'seconds')

drop columns: 5.405134201049805 seconds


In [3]:
tweets_random.head()

,tweet_id,text
0,367881326273105920,@shoebydoo32 I only left to go back home for t...
1,367881326281519105,oh my god bina id idnt read that u played GTA ...
2,367882185916702722,I Have To Make What I Think Is The Best Decisi...
3,367883553121394690,@elizrod_ that's from hard work 😉
4,367883867719348224,RT @MileenaSucks: Can I just lay out in the gr...


# filtered top 10M files

In [8]:
print(merged_filtered.shape)

(65461336, 3)


In [9]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


# boundary_list = [i/20.0 for i in range(11)]
# boundary_list.reverse()
# print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
#     if column == 'job_offer': continue
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_filtered = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'filtered*.csv')], ignore_index = True)
    print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds', model_output_filtered.shape)
    
    start_time = time.time()
    model_output_filtered = model_output_filtered[['tweet_id', 'proba']]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    start_time = time.time()
    merged_filtered = pd.merge(model_output_filtered, tweets_filtered, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)

    print(merged_filtered.shape)

    start_time = time.time()
    merged_filtered = merged_filtered.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
    
    start_time = time.time()
    merged_filtered = merged_filtered.sort_values(by=['proba'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_filtered.shape)    
    
    start_time = time.time()
    merged_filtered_temp = merged_filtered[:5000000]
    print('top 5M truncate:', str(time.time() - start_time), 'seconds', merged_filtered.shape)   
    
    print(merged_filtered_temp.tail(50)['text'])
    
    start_time = time.time()
    pickle.dump( merged_filtered_temp, open( "./ONNXturk100M/{}_GLOVE_CNN_merged_filtered_100m_jun22.pkl".format(column), "wb" ) )
    print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break




 is_unemployed
/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/is_unemployed/
time taken to load filtered sample: 26.308045148849487 seconds (40809972, 2)
drop columns: 0.28781652450561523 seconds
time taken to merge: 77.44315648078918 seconds (40809972, 3)
(40809972, 3)
drop dup: 55.0835325717926 seconds (36333272, 3)
sort: 22.951133966445923 seconds (36333272, 3)
top 5M truncate: 2.2111856937408447 seconds (36333272, 3)
24732266    #JobSearch Brand Ambassador XM - Brand Activat...
34812390    Best Boston Careers CFO Cambridge Operating Gr...
1178028     RT @OldSpice: Ray Lewis taught us real success...
6276315     When your coworker plays your favorite song at...
22157731    Illinois New Jobs $ Company Truck Drivers - Fl...
1569147     Las Vegas New Jobs $ Pharmaceutical Customer S...
21933438    New #job opening at Richemont in #Houston - #B...
1865894     Computers &amp; Networking #5 : http://t.co/cA...
2275735     Telecommute Jobs $$$ Bu

time taken to dump: 3.8571767807006836 seconds


 is_hired_1mo
/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/is_hired_1mo/
time taken to load filtered sample: 30.5049307346344 seconds (40809972, 2)
drop columns: 0.29251646995544434 seconds
time taken to merge: 77.83843755722046 seconds (40809972, 3)
(40809972, 3)
drop dup: 54.788901567459106 seconds (36333272, 3)
sort: 23.370448112487793 seconds (36333272, 3)
top 5M truncate: 2.6819756031036377 seconds (36333272, 3)
39013089    @flocktard @seanhannity @UrbanAchievr What are...
14786170    @chiu0nthls no significant network problems th...
3386967     Hypnotist helps woman quit smoking - KPRC Hous...
31963888    Sen. Jeff Sessions Sends Memo Urging GOP Colle...
15147608    RT @sallykohn: Please please please please PLE...
29040816    HOW ANY MAJOR ELECTION REALLY WORKS: https://t...
1105230     Fired Texas trooper who probed women's genital...
9308064     TEENAGERS HATE: ? Homework ? Fake friend

In [ ]:
start_time = time.time()
merged_filtered_temp = merged_filtered[5000000:12000000]
print('top 5M truncate:', str(time.time() - start_time), 'seconds', merged_filtered.shape)   

# print(merged_filtered_temp.tail(50)['text'])

start_time = time.time()
pickle.dump( merged_filtered_temp, open( "./ONNXturk100M/{}_GLOVE_CNN_merged_filtered_100m_jun22_5_to_12M.pkl".format(column), "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')



top 5M truncate: 0.0004470348358154297 seconds (36333272, 3)
333802      @USPS Bang up job so far guys.. Think you miss...
40415006                    RT @Obey_Katii: Instagram working
12124043    Too much sweetness in a relationship leads to ...
4671441     Extremely humbled &amp; honored to announce I ...
27341155    RT @cl_atlanta: Atlanta jobs agency gets full-...
18499039    RT @JudithKBrennan: @HellgateOsprey The nest i...
25467470    Frustrations of a Job Seeker by Ed Tsyitee Jr ...
35856640    "@LamontBender: Cooking and Working Out! #myth...
20454317    Prayer and devotion are a great way to start t...
9798804     Quit being cheap Oakland and build another sta...
30134276    RT @3Mpowerment: Ladies and Gentlemen Please W...
2175210             @Pabloowwjobb si ahorita voy. Yeah right!
8356666     #AbandonedTreasures Jan 2016 #Vintiques #Paint...
22128914    Give your ego the day off. Your job today is t...
39647160    Oh hey, I got an interview for a job at Target...
27483899 

# random top 10M files

In [11]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


# boundary_list = [i/20.0 for i in range(11)]
# boundary_list.reverse()
# print('boundary_list', boundary_list)

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    if column != 'job_offer': continue
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_random = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'random*.csv')], ignore_index = True)
    print('time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random.shape)
    
    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'proba']]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    start_time = time.time()
    merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
    print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)

#     print(merged_random.head())

    start_time = time.time()
    merged_random = merged_random.drop_duplicates(['text'])
    print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)
    
    start_time = time.time()
    merged_random = merged_random.sort_values(by=['proba'], ascending=False)
    print('sort:', str(time.time() - start_time), 'seconds', merged_random.shape)    
    
#     start_time = time.time()
#     merged_random_temp = merged_random[:5000000]
#     print('top 5M truncate:', str(time.time() - start_time), 'seconds', merged_random.shape)   
    
#     print(merged_random_temp.tail(50)['text'])
    
#     start_time = time.time()
#     pickle.dump( merged_random_temp, open( "./ONNXturk100M/{}_GLOVE_CNN_merged_random_100m_jun22.pkl".format(column), "wb" ) )
#     print('time taken to dump:', str(time.time() - start_time), 'seconds')
        
#     break




 is_unemployed


 lost_job_1mo


 job_search


 is_hired_1mo


 job_offer
/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/job_offer/
time taken to load random sample: 66.41552114486694 seconds (91210308, 2)
drop columns: 0.9590597152709961 seconds
time taken to merge: 99.23714971542358 seconds (91210308, 3)
drop dup: 117.09680914878845 seconds (84034162, 3)
sort: 59.81334638595581 seconds (84034162, 3)


In [ ]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_filtered = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'filtered*.csv')], ignore_index = True)
    print('time taken to load filtered sample:', str(time.time() - start_time), 'seconds', model_output_filtered.shape)
    
    start_time = time.time()
    model_output_filtered = model_output_filtered[['tweet_id', 'proba']]
    model_output_filtered.columns = ['tweet_id', column]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    if column == 'is_unemployed':
        start_time = time.time()
        merged_filtered = pd.merge(model_output_filtered, tweets_filtered, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
    else:
        start_time = time.time()
        merged_filtered = pd.merge(model_output_filtered, merged_filtered, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_filtered.shape)
            
    print(merged_filtered.tail(10))
            
#     break
    
    
start_time = time.time()
merged_filtered = merged_filtered.drop_duplicates(['text'])
print('drop dup:', str(time.time() - start_time), 'seconds', merged_filtered.shape)

start_time = time.time()
pickle.dump( merged_filtered, open( "./ONNXturk100M/ALL_ONNX_BERT_ST_merged_filtered_100m_jun24.pkl", "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')
        


# merging random tweets into one big file

In [ ]:
# outputting 1M (pinky finger up) tweets by column
import glob
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np


for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:
    print('\n\n', column)
    
    start_time = time.time()
    model_output_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/pred_output_100M_GLOVE_CNN_optimized/{}/'.format(column)
    print(model_output_path)

    start_time = time.time()
    model_output_random = pd.concat([pd.read_csv(f) for f in glob.glob(model_output_path+'random*.csv')], ignore_index = True)
    print('time taken to load random sample:', str(time.time() - start_time), 'seconds', model_output_random.shape)
    
    start_time = time.time()
    model_output_random = model_output_random[['tweet_id', 'proba']]
    model_output_random.columns = ['tweet_id', column]
    print('drop columns:', str(time.time() - start_time), 'seconds')
    

    if column == 'is_unemployed':
        start_time = time.time()
        merged_random = pd.merge(model_output_random, tweets_random, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)
    else:
        start_time = time.time()
        merged_random = pd.merge(model_output_random, merged_random, how='inner', on = 'tweet_id')
        print('time taken to merge:', str(time.time() - start_time), 'seconds', merged_random.shape)
            
    print(merged_random.tail(10))
            
#     break
    
    
start_time = time.time()
merged_random = merged_random.drop_duplicates(['text'])
print('drop dup:', str(time.time() - start_time), 'seconds', merged_random.shape)

start_time = time.time()
pickle.dump( merged_random, open( "./ONNXturk100M/ALL_GLOVE_CNN_merged_random_100m_jun24.pkl", "wb" ) )
print('time taken to dump:', str(time.time() - start_time), 'seconds')
        


In [ ]:
model_output_random.head()